In [50]:
import requests
import json
import pandas as pd
import numpy as np
from pymongo import MongoClient
import math
from bs4 import BeautifulSoup
import folium
from folium import Choropleth, Circle, Marker
from folium.plugins import HeatMap, MarkerCluster
#import src.functions as fn

In [2]:
client = MongoClient("mongodb://localhost/companies")
db = client.get_database()

## Indice calidad de vida

En primer lugar, voy a basarme en el índice de calidad de vida para seleccionar una zona. 

El índice de calidad de vida se calcula en base otros índices: índice de poder adquisitivo, índice de seguridad, indice de sanidad, de costo de visa de tiempo de desplazamiento den tráfico, contaminación clima y relación precio/ingresos para propiedades.

Para obtener esta información he hecho scrapping de la web https://es.numbeo.com/calidad-de-vida/clasificaciones-por-pa%C3%ADs.

In [3]:
data=requests.get("https://es.numbeo.com/calidad-de-vida/clasificaciones-por-pa%C3%ADs").text
soup= BeautifulSoup(data, 'html.parser')

def procesaIndices(fila):
    m = fila.find_all("td")
    #print(m[2])
    return {
        "country":m[1].text.strip(),
        "calidad_vida":float((m[2].text).replace(',','.')),
        "poder_adquisitivo":float((m[3].text).replace(',','.')),
        "seguridad":float((m[4].text).replace(',','.')),
        "sanidad":float((m[5].text).replace(',','.')),
        "costo_vida":float((m[6].text).replace(',','.')),
        "relacion_precio_vs_ingresos":float((m[7].text).replace(',','.')),
        "tiempo_desplazamiento":float((m[8].text).replace(',','.')),
        "contaminación":float((m[9].text).replace(',','.')),
        "clima":float((m[10].text).replace(',','.'))
    }
    
indice = soup.find_all('table')[2]
indice_dict = [procesaIndices(fila) for fila in indice.find_all("tr")[1:]]


In [4]:
#Genero df:
df_indices=pd.DataFrame(indice_dict)

#Exporto en formato json y luego importo en MongoDB Compass:
df_indices.to_json("../output/indices_calidad.json", orient="records")

El país que elegiré para poner la nueva sede, será aquel con mejor indice de calidad y además con un indice de clima superior a 90. Como se ve a continuación será Australia.

In [5]:
df_indices[(df_indices.clima>90)].head()

,country,calidad_vida,poder_adquisitivo,seguridad,sanidad,costo_vida,relacion_precio_vs_ingresos,tiempo_desplazamiento,contaminación,clima
3,Australia,186.21,107.31,58.64,77.38,73.54,7.52,34.73,23.46,92.70
7,Nueva Zelanda,181.02,92.66,59.07,73.81,72.53,8.52,31.10,23.40,95.46
14,España,169.82,72.03,68.04,78.88,53.77,9.37,29.10,39.99,94.19
18,Portugal,162.91,49.43,70.37,71.88,49.52,12.70,30.00,30.89,97.31
25,Francia,153.95,80.36,53.21,79.99,74.14,13.04,34.76,43.56,90.25


## Importo de MongoDB Compass:
Con una query me traeré de la colección 'companies' todas las empresas que tenga una ganancia distinta de 0$.

In [6]:
q1={"total_money_raised": {"$ne": "$0"}}
companies=list(db["companies"].find(q1,{"name":1,"founded_year":1,"total_money_raised":1, "offices":1,"category_code":1}))

In [7]:
df = pd.DataFrame(companies)

Voy a tratar el df para desagregar la columna offices, dejando cada oficina en una línea distina y luego desdoblando las columnas que contienen la información de cada oficina.

In [8]:
#Desagrego las oficinas, para que por compañía cada oficina aparezca en una línea independiente:
df = df.explode('offices')
display(df.head())

#Desagrego la columna office para obtener los datos en columnas de cada oficina:
df_offices = df[["offices"]].apply(lambda x: x.offices, result_type="expand", axis=1)
display(df_offices.head())

#Unifico los dos df anteriores para tener un único df completo con toda la información
clean_data = pd.concat([df,df_offices], axis=1)
display(clean_data.head())

,_id,name,category_code,founded_year,total_money_raised,offices
0,52cdef7c4bab8bd675297d8f,Omnidrive,network_hosting,2005.0,$800k,"{'description': '', 'address1': 'Suite 200', '..."
1,52cdef7c4bab8bd675297d91,Geni,web,2006.0,$16.5M,"{'description': 'Headquarters', 'address1': '9..."
2,52cdef7c4bab8bd675297d8a,Wetpaint,web,2005.0,$39.8M,"{'description': '', 'address1': '710 - 2nd Ave..."
2,52cdef7c4bab8bd675297d8a,Wetpaint,web,2005.0,$39.8M,"{'description': '', 'address1': '270 Lafayette..."
3,52cdef7c4bab8bd675297d96,Gizmoz,web,2003.0,$18.1M,"{'description': None, 'address1': None, 'addre..."


,description,address1,address2,zip_code,city,state_code,country_code,latitude,longitude
0,,Suite 200,654 High Street,94301,Palo Alto,CA,ISR,NaN,NaN
1,Headquarters,9229 W. Sunset Blvd.,,90069,West Hollywood,CA,USA,34.090368,-118.393064
2,,710 - 2nd Avenue,Suite 1100,98104,Seattle,WA,USA,47.603122,-122.333253
2,,270 Lafayette Street,Suite 505,10012,New York,NY,USA,40.723731,-73.996431
3,None,None,None,None,Menlo Park,CA,USA,37.484130,-122.169472


,_id,name,category_code,founded_year,total_money_raised,offices,description,address1,address2,zip_code,city,state_code,country_code,latitude,longitude
0,52cdef7c4bab8bd675297d8f,Omnidrive,network_hosting,2005.0,$800k,"{'description': '', 'address1': 'Suite 200', '...",,Suite 200,654 High Street,94301,Palo Alto,CA,ISR,NaN,NaN
1,52cdef7c4bab8bd675297d91,Geni,web,2006.0,$16.5M,"{'description': 'Headquarters', 'address1': '9...",Headquarters,9229 W. Sunset Blvd.,,90069,West Hollywood,CA,USA,34.090368,-118.393064
2,52cdef7c4bab8bd675297d8a,Wetpaint,web,2005.0,$39.8M,"{'description': '', 'address1': '710 - 2nd Ave...",,710 - 2nd Avenue,Suite 1100,98104,Seattle,WA,USA,47.603122,-122.333253
2,52cdef7c4bab8bd675297d8a,Wetpaint,web,2005.0,$39.8M,"{'description': '', 'address1': '270 Lafayette...",,270 Lafayette Street,Suite 505,10012,New York,NY,USA,40.723731,-73.996431
3,52cdef7c4bab8bd675297d96,Gizmoz,web,2003.0,$18.1M,"{'description': None, 'address1': None, 'addre...",None,None,None,None,Menlo Park,CA,USA,37.484130,-122.169472


Elimino la columna _id, ya que se generará un nuevo id cuando cargue la nueva colección, y también elimino office que ya está toda la información desagregada y no nos sirve más.

In [9]:
clean_data = clean_data.drop(columns=["_id","offices"])
clean_data.head()

,name,category_code,founded_year,total_money_raised,description,address1,address2,zip_code,city,state_code,country_code,latitude,longitude
0,Omnidrive,network_hosting,2005.0,$800k,,Suite 200,654 High Street,94301,Palo Alto,CA,ISR,NaN,NaN
1,Geni,web,2006.0,$16.5M,Headquarters,9229 W. Sunset Blvd.,,90069,West Hollywood,CA,USA,34.090368,-118.393064
2,Wetpaint,web,2005.0,$39.8M,,710 - 2nd Avenue,Suite 1100,98104,Seattle,WA,USA,47.603122,-122.333253
2,Wetpaint,web,2005.0,$39.8M,,270 Lafayette Street,Suite 505,10012,New York,NY,USA,40.723731,-73.996431
3,Gizmoz,web,2003.0,$18.1M,None,None,None,None,Menlo Park,CA,USA,37.484130,-122.169472


Con la siguiente función, daré el formato GeoJson, necesario para poder generar los indexes en MongoDB Compass. En aquellos casos en los que latitud o longitud sean valores NaN, se indicará None, para evitar posibles conflicos en la carga de la colección.

In [10]:
def asGeoJSON(lat,lng):
    try:
        lat = float(lat)
        lng = float(lng)
        if not math.isnan(lat) and not math.isnan(lng):
            return {
                "type":"Point",
                "coordinates":[lng,lat]
            }
    except Exception:
        print("Invalid data")
        return None
        

clean_data["location"] = clean_data[["latitude","longitude"]].apply(lambda x:asGeoJSON(x.latitude,x.longitude), axis=1)
clean_data[["latitude","longitude","location"]].head()

,latitude,longitude,location
0,NaN,NaN,None
1,34.090368,-118.393064,"{'type': 'Point', 'coordinates': [-118.393064,..."
2,47.603122,-122.333253,"{'type': 'Point', 'coordinates': [-122.333253,..."
2,40.723731,-73.996431,"{'type': 'Point', 'coordinates': [-73.9964312,..."
3,37.484130,-122.169472,"{'type': 'Point', 'coordinates': [-122.169472,..."


Añado una columna que tipifique si la empresa tiene 10 o menos años('young') o más de 10 años ('old').

In [11]:
def oldYoung(row):
    if row['founded_year']>=2007:
        return 'young'
    else:
        return 'old'

clean_data['old_young']= clean_data.apply(lambda row: oldYoung(row), axis=1)

clean_data[['name', 'founded_year','old_young']].head(10)

,name,founded_year,old_young
0,Omnidrive,2005.0,old
1,Geni,2006.0,old
2,Wetpaint,2005.0,old
2,Wetpaint,2005.0,old
3,Gizmoz,2003.0,old
4,Lala,NaN,old
5,StumbleUpon,2002.0,old
5,StumbleUpon,2002.0,old
6,Slacker,2006.0,old
7,Scribd,2007.0,young


Exporto a json y luego lo importo a MongoDB Compass.

In [12]:
clean_data.to_json("../output/cleaned_offices.json", orient="records")

Lo importo en Mongo DB Compass: $ mongoimport --db companies --collection offices --jsonArray --drop cleaned_offices.json

Una vez importado, creo un geospartial index: Indexes > Create Index > (write any indexname) > Select fieldname:"location" and 2dsphere

## Importo de MongoDB:
Ahora importo las oficinas que están en Australia para ver en un mapa su distribución

In [19]:
q2={"$and": [{"country_code": {"$eq": "AUS"}}, {"location": {"$ne":None}}]}
aus = list(db["offices"].find(q2, {'_id':0,'name':1, 'total_money_raised':1,'category_code':1, 'founded_year':1, 'city':1, 'latitude':1, 'longitude':1, 'old_young':1}))
df_aus = pd.DataFrame(aus)
df_aus.head()

,name,category_code,founded_year,total_money_raised,city,latitude,longitude,old_young
0,Google,search,1998.0,$555M,Melbourne,-37.879234,145.073608,old
1,Google,search,1998.0,$555M,Sydney,-34.822723,138.612396,old
2,Hitwise,web,NaN,$4.2M,Melbourne,-37.814251,144.963169,old
3,2threads,games_video,2004.0,$300k,sydney,-33.884685,151.216427,old
4,99designs,design,2008.0,$35M,Collingwood,-37.802659,144.986855,young


## Mapa en folium:

Antes consultaré las coordenadas de Australia en geoCode con la siguiente función:

In [14]:
def geocode(address):
    data = requests.get(f"https://geocode.xyz/{address}?json=1").json()
    print(data)
    return {
        "type":"Point",
        "coordinates":[float(data["longt"]),float(data["latt"])]
    }

aus_geo = geocode("Melbourn")

{'standard': {'addresst': {}, 'city': 'Melbourn', 'prov': 'UK', 'countryname': 'United Kingdom', 'postal': {}, 'confidence': '0.90'}, 'longt': '0.02860', 'alt': {'loc': {'longt': '0.01950', 'prov': 'UK', 'city': 'Melbourn', 'countryname': 'United Kingdom', 'postal': 'SG8', 'region': {}, 'latt': '52.08366'}}, 'elevation': {}, 'latt': '52.09674'}


In [15]:
start_lat = -25.2743988
start_lon = 133.7751312
aus_map = folium.Map(location=[start_lat, start_lon],tiles='cartodbpositron', zoom_start=4)
aus_map

In [16]:
df_aus.apply(lambda row:folium.Marker(location=[row["latitude"], 
                                                row["longitude"]], popup=row["old_young"]).add_to(aus_map),axis=1)
aus_map

Genero query para traeme los aeropuertos que hay en Australia

In [17]:
q3={"$and": [{"country": {"$eq":"Australia"}}, {"$or":[{"city":{"$eq":"Sydney"}},{"city":{"$eq":"Melbourne"}}]}]}
aus_airport = list(db['airports'].find(q3,{"_id":0, "name":1,"city":1,"lat":1,"lon":1}))

df_airports = pd.DataFrame (aus_airport)
df_airports.head()

,lat,lon,name,city
0,-37.6759,144.844,Melbourne International Airport,Melbourne
1,-33.9344,151.168,Kingsford Smith Airport,Sydney
2,-33.9344,151.168,Kingsford Smith Airport,Sydney


In [18]:
df_airports.apply(lambda row:folium.Marker(location=[row["lat"], 
                                                row["lon"]], popup=row["name"],icon=folium.Icon(color='black', icon_color='white', icon='plane')).add_to(aus_map),axis=1)
aus_map

## Colsultas a GooglePlace: starbucks, pubs, parques...

Request a la API de google consultando los starbucks en sydney.

In [39]:
'''import os
from dotenv import load_dotenv
load_dotenv('src/.env')

def requestGooglePlace(place,token):
    #token = os.getenv("API_GOOGLE_KEY")
    #print(token)
    if not token:
        raise ValueError("Necesitas un API_GOOGLE_KEY")
    
    baseUrl = "https://maps.googleapis.com/maps/api/place/textsearch/json?query="
    url = baseUrl+place+"&key="
    print(f"Requesting data from {url}")
    headers = {
        "Authorization": f"token {token}"
    }
    res = requests.get(url,headers=headers)
    if res.status_code != 200:
        print(res.text)
        raise ValueError("Bad Response")
    return res.json()

data_Starbucks=requestGooglePlace("Starbucks+in+Sydney")'''

Requesting data from https://maps.googleapis.com/maps/api/place/textsearch/json?query=Starbucks+in+Sydney&key=


In [143]:
#data_Starbucks=requests.get("https://maps.googleapis.com/maps/api/place/textsearch/json?query=Starbucks+in+Sydney&key=<<AQUÍ EL TOKEN>>")

Convierto a json y luego extraigo sólo la key "results" que es donde almacena todos los Starbucks.

In [82]:
data_Starbucks_sydney=data_Starbucks.json()
results=data_Starbucks_sydney['results']
json_Starbucks_sydney=json.dumps(results)

Exporto el json para luego poderlo importar a MongoDB

In [96]:
writeFile =open('starbucks_sydney.json', 'w')
writeFile.write(json_Starbucks_sydney)
writeFile.close()

Con esta query voy a poder extraer mejor la localización, para luego agragarlo al mapa:

In [116]:
q4={"name": {"$eq":"Starbucks"}}
syd_starbucks = list(db['starbucks_sydney'].find(q4,{"_id":0, "name":1,"geometry.location":1,}))
df_syd_starbucks=pd.DataFrame(syd_starbucks)


In [129]:
#Desagrego la columna geometry.location para obtener la latitud y longitud en columnas de cada Starbucks:
df_loc_syd_cafe = df_syd_starbucks[["geometry"]].apply(lambda x: x.geometry, result_type="expand", axis=1)

df_loc_syd_cafe2 = df_loc_syd_cafe[["location"]].apply(lambda x: x.location, result_type="expand", axis=1)
display(df_loc_syd_cafe2.head())

#Unifico los dos df anteriores para tener un único df completo con toda la información
clean_starbucks_sydney = pd.concat([df_syd_starbucks,df_loc_syd_cafe,df_loc_syd_cafe2], axis=1)
display(clean_starbucks_sydney.head())

#Elimino columnas geometry y location:
clean_starbucks_sydney.drop(['geometry', 'location'], axis=1, inplace=True)

display(clean_starbucks_sydney.head())

,lat,lng
0,-33.939276,151.165472
1,-33.766014,150.817496
2,-33.873384,151.209596
3,-33.872376,151.206620
4,-33.879241,151.205680


,geometry,name,location,lat,lng
0,"{'location': {'lat': -33.9392765, 'lng': 151.1...",Starbucks,"{'lat': -33.9392765, 'lng': 151.1654724}",-33.939276,151.165472
1,"{'location': {'lat': -33.7660145, 'lng': 150.8...",Starbucks,"{'lat': -33.7660145, 'lng': 150.8174962}",-33.766014,150.817496
2,"{'location': {'lat': -33.873384, 'lng': 151.20...",Starbucks,"{'lat': -33.873384, 'lng': 151.209596}",-33.873384,151.209596
3,"{'location': {'lat': -33.8723764, 'lng': 151.2...",Starbucks,"{'lat': -33.8723764, 'lng': 151.2066203}",-33.872376,151.206620
4,"{'location': {'lat': -33.8792412, 'lng': 151.2...",Starbucks,"{'lat': -33.8792412, 'lng': 151.2056795}",-33.879241,151.205680


,name,lat,lng
0,Starbucks,-33.939276,151.165472
1,Starbucks,-33.766014,150.817496
2,Starbucks,-33.873384,151.209596
3,Starbucks,-33.872376,151.206620
4,Starbucks,-33.879241,151.205680


In [131]:
clean_starbucks_sydney["location"] = clean_starbucks_sydney[["lat","lng"]].apply(lambda x:asGeoJSON(x.lat,x.lng), axis=1)
clean_starbucks_sydney[["lat","lng","location"]].head()

,lat,lng,location
0,-33.939276,151.165472,"{'type': 'Point', 'coordinates': [151.1654724,..."
1,-33.766014,150.817496,"{'type': 'Point', 'coordinates': [150.8174962,..."
2,-33.873384,151.209596,"{'type': 'Point', 'coordinates': [151.209596, ..."
3,-33.872376,151.206620,"{'type': 'Point', 'coordinates': [151.2066203,..."
4,-33.879241,151.205680,"{'type': 'Point', 'coordinates': [151.2056795,..."


In [132]:
clean_starbucks_sydney.to_json("../output/clean_starbucks_sydney.json", orient="records")

## Incluyo en el mapa los Starbucks de Sydney

In [142]:
clean_starbucks_sydney.apply(lambda row:folium.Marker(location=[row["lat"], 
                                                row["lng"]], popup=row["name"],icon=folium.Icon(color='red', icon_color='white')).add_to(aus_map),axis=1)
aus_map